In [1]:
!pip install -q unsloth transformers accelerate peft


In [ ]:
from unsloth import FastLanguageModel
import torch

# -----------------------------
# Load Model from HuggingFace
# -----------------------------
HF_MODEL_ID = "anonym-author/llama3-8b-instruct-arabic"

print("Loading model from HuggingFace...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=HF_MODEL_ID,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,  # Use 4-bit to save memory in Colab
)

# Enable inference mode
FastLanguageModel.for_inference(model)
model.eval()
print("Model loaded successfully!")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Loading model from HuggingFace...
==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model loaded successfully!


In [3]:
def isTrigger(text: str):
    """
    Classify patient statement as trigger (1) or non-trigger (0).

    Args:
        text: Patient statement in English or Arabic

    Returns:
        dict with 'label' and 'confidence'
    """

    prompt = (
        "### Instruction:\n"
        "Classify the following patient statement as 0 (Not Triggered) or 1 (Triggered).\n"
        "Respond with exactly one character: 0 or 1.\n\n"
        "### Statement:\n"
        f"{text}\n\n"
        "### Response:\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Get logits instead of generating
    with torch.no_grad():
        outputs = model(input_ids=inputs.input_ids, attention_mask=inputs.attention_mask)
        logits = outputs.logits[0, -1, :]  # Logits for the next token

        # Get token IDs for "0" and "1"
        token_0 = tokenizer.encode("0", add_special_tokens=False)[0]
        token_1 = tokenizer.encode("1", add_special_tokens=False)[0]

        # Get probabilities for these tokens
        probs = torch.softmax(logits, dim=-1)
        prob_0 = probs[token_0].item()
        prob_1 = probs[token_1].item()

        # Predict based on which has higher probability
        if prob_1 > prob_0:
            predicted_label = 1
            confidence = prob_1
        else:
            predicted_label = 0
            confidence = prob_0

    label_map = {0: "Non-trigger", 1: "Trigger"}

    result = {
        "label": label_map[predicted_label],
        "confidence": confidence,
        "predicted_digit": predicted_label
    }

    return result


In [4]:
# -----------------------------
# Diagnostic Code (Fixed)
# -----------------------------

test_statements = [
    "I'm feeling much better now",
    "Everything is fine",
    "No problems at all",
    "I feel great",
    "I have a severe headache and dizziness",
    "I'm experiencing chest pain",
    "لقد أصبت بطفح جلدي شديد أثناء تناول الدواء.",
    "I'm not feeling well",
    "The medication is making me sick",
    "أنا بخير، شكراً لك",  # I'm fine, thank you
    "كل شيء على ما يرام",  # Everything is okay
    "أشعر بتحسن كبير",  # I feel much better
    "لا توجد مشاكل",  # No problems
    "الدواء يعمل بشكل جيد",  # The medication is working well
    "أشعر بصحة جيدة",  # I feel healthy
    "لم أعد أشعر بالألم",  # I no longer feel pain
    "تعافيت تماماً",  # I've recovered completely
    "أشعر بتحسن كل يوم",  # I feel better every day
    "لا أعاني من أي أعراض",  # I don't have any symptoms
    "صحتي ممتازة الآن",  # My health is excellent now
    "الأمور تسير على ما يرام",  # Things are going well
    "أنا أفضل بكثير الآن",  # I'm much better now
    "لا يوجد ألم",  # There is no pain
    "أشعر بالراحة",  # I feel comfortable
]

print("\n" + "="*60)
print("Testing a few statements:")
print("="*60)

for statement in test_statements:
    prompt = (
        "### Instruction:\n"
        "Classify the following patient statement as 0 (Not Triggered) or 1 (Triggered).\n"
        "Respond with exactly one character: 0 or 1.\n\n"
        "### Statement:\n"
        f"{statement}\n\n"
        "### Response:\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        # Generate with output_scores to get probabilities
        output = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=1,
            do_sample=False,
            return_dict_in_generate=True,
            output_scores=True
        )

        output_ids = output.sequences
        scores = output.scores[0][0]  # Logits for the first (and only) generated token

        predicted_token = tokenizer.decode(
            output_ids[0, inputs.input_ids.shape[1]:],
            skip_special_tokens=True
        ).strip()

        # Get token IDs for "0" and "1"
        token_0 = tokenizer.encode("0", add_special_tokens=False)[0]
        token_1 = tokenizer.encode("1", add_special_tokens=False)[0]

        # Get probabilities
        probs = torch.softmax(scores, dim=-1)
        prob_0 = probs[token_0].item()
        prob_1 = probs[token_1].item()

    print(f"\nStatement: {statement}")
    print(f"Generated: '{predicted_token}'")
    print(f"P(0) = {prob_0:.4f}, P(1) = {prob_1:.4f}")
    if prob_0 > 0:
        print(f"Ratio P(1)/P(0) = {prob_1/prob_0:.2f}")


Testing a few statements:

Statement: I'm feeling much better now
Generated: '1'
P(0) = 0.3207, P(1) = 0.6789
Ratio P(1)/P(0) = 2.12

Statement: Everything is fine
Generated: '0'
P(0) = 0.6222, P(1) = 0.3774
Ratio P(1)/P(0) = 0.61

Statement: No problems at all
Generated: '0'
P(0) = 0.5924, P(1) = 0.4072
Ratio P(1)/P(0) = 0.69

Statement: I feel great
Generated: '1'
P(0) = 0.4376, P(1) = 0.5619
Ratio P(1)/P(0) = 1.28

Statement: I have a severe headache and dizziness
Generated: '1'
P(0) = 0.0534, P(1) = 0.9465
Ratio P(1)/P(0) = 17.73

Statement: I'm experiencing chest pain
Generated: '1'
P(0) = 0.4072, P(1) = 0.5925
Ratio P(1)/P(0) = 1.45

Statement: لقد أصبت بطفح جلدي شديد أثناء تناول الدواء.
Generated: '1'
P(0) = 0.0675, P(1) = 0.9323
Ratio P(1)/P(0) = 13.80

Statement: I'm not feeling well
Generated: '1'
P(0) = 0.1480, P(1) = 0.8517
Ratio P(1)/P(0) = 5.75

Statement: The medication is making me sick
Generated: '1'
P(0) = 0.0758, P(1) = 0.9239
Ratio P(1)/P(0) = 12.18

Statement: أنا

In [5]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
CSV_PATH = "/content/drive/MyDrive/Memo_Dataset.csv"

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [8]:
df = pd.read_csv(CSV_PATH)
# keep only columns we need
#df = df[['Question', 'Trigger']].dropna() #arabic column
df = df[['Question_eng', 'Trigger']].dropna() #english column
#df = df.rename(columns={'Question': 'text', 'Trigger': 'label'})
df = df.rename(columns={'Question_eng': 'text', 'Trigger': 'label'})
df['label'] = df['label'].astype(int)

print("Raw dataset size:", len(df))
print(df['label'].value_counts())

# Balance classes by downsampling the majority class
min_count = df['label'].value_counts().min()
df_bal = df.groupby('label', group_keys=False).apply(lambda x: x.sample(min_count, random_state=42)).reset_index(drop=True)
print("Balanced dataset size:", len(df_bal))
print(df_bal['label'].value_counts())

train_df, test_df = train_test_split(df_bal, test_size=0.2, random_state=42, stratify=df_bal['label'])
print(f"Train: {len(train_df)}, Test: {len(test_df)}")

Raw dataset size: 13142
label
1    7687
0    5455
Name: count, dtype: int64
Balanced dataset size: 10910
label
0    5455
1    5455
Name: count, dtype: int64
Train: 8728, Test: 2182


/tmp/ipython-input-4211767844.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_bal = df.groupby('label', group_keys=False).apply(lambda x: x.sample(min_count, random_state=42)).reset_index(drop=True)


In [9]:
print(test_df['label'].value_counts())

label
0    1091
1    1091
Name: count, dtype: int64


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from datetime import datetime
import torch

# -----------------------------
# isTrigger function with probabilities
# -----------------------------
def isTrigger(text: str):
    """
    Classify patient statement as trigger (1) or non-trigger (0).

    Args:
        text: Patient statement in English or Arabic

    Returns:
        dict with 'label' (0 or 1) and 'confidence' (probability)
    """

    prompt = (
        "### Instruction:\n"
        "Classify the following patient statement as 0 (Not Triggered) or 1 (Triggered).\n"
        "Respond with exactly one character: 0 or 1.\n\n"
        "### Statement:\n"
        f"{text}\n\n"
        "### Response:\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        # Generate with scores
        output = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=1,
            do_sample=False,
            return_dict_in_generate=True,
            output_scores=True
        )

        output_ids = output.sequences
        scores = output.scores[0][0]  # Logits for the generated token

        predicted_token = tokenizer.decode(
            output_ids[0, inputs.input_ids.shape[1]:],
            skip_special_tokens=True
        ).strip()

        # Get token IDs for "0" and "1"
        token_0 = tokenizer.encode("0", add_special_tokens=False)[0]
        token_1 = tokenizer.encode("1", add_special_tokens=False)[0]

        # Get probabilities
        probs = torch.softmax(scores, dim=-1)
        prob_0 = probs[token_0].item()
        prob_1 = probs[token_1].item()

    # Extract predicted label
    predicted_label = None
    for ch in predicted_token:
        if ch in ("0", "1"):
            predicted_label = int(ch)
            break

    if predicted_label is None:
        print(f"Warning: Model generated '{predicted_token}' instead of 0 or 1")
        predicted_label = 0

    # Get confidence for the predicted label
    confidence = prob_0 if predicted_label == 0 else prob_1

    result = {
        "label": predicted_label,
        "confidence": confidence,
        "prob_0": prob_0,
        "prob_1": prob_1
    }

    return result

# -----------------------------
# Evaluate on test set
# -----------------------------
print("="*60)
print(f"Evaluating on test set ({len(test_df)} samples)")
print("="*60)

predictions = []
confidences = []
prob_0_list = []
prob_1_list = []

for idx, row in test_df.iterrows():
    statement = row['text']
    result = isTrigger(statement)
    predictions.append(result['label'])
    confidences.append(result['confidence'])
    prob_0_list.append(result['prob_0'])
    prob_1_list.append(result['prob_1'])

    if (idx + 1) % 10 == 0:
        print(f"Processed {idx + 1}/{len(test_df)} samples...")

# Create results dataframe
test_results_df = test_df.copy()
test_results_df['prediction'] = predictions
test_results_df['confidence'] = confidences
test_results_df['prob_0'] = prob_0_list
test_results_df['prob_1'] = prob_1_list
test_results_df['correct'] = test_results_df['label'] == test_results_df['prediction']

# Calculate metrics
accuracy = accuracy_score(test_results_df['label'], predictions)
precision = precision_score(test_results_df['label'], predictions)
recall = recall_score(test_results_df['label'], predictions)
f1 = f1_score(test_results_df['label'], predictions)

print("\n" + "="*60)
print("TEST SET RESULTS:")
print("="*60)
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

# Confusion matrix
print("\n" + "="*60)
print("Confusion Matrix:")
print("="*60)
cm = confusion_matrix(test_results_df['label'], predictions)
print(f"                Predicted 0  Predicted 1")
print(f"Actual 0        {cm[0][0]:>11}  {cm[0][1]:>11}")
print(f"Actual 1        {cm[1][0]:>11}  {cm[1][1]:>11}")

# Classification report
print("\n" + "="*60)
print("Classification Report:")
print("="*60)
print(classification_report(test_results_df['label'], predictions,
                          target_names=['Non-trigger (0)', 'Trigger (1)']))

# Average confidence for correct vs incorrect predictions
print("\n" + "="*60)
print("Confidence Analysis:")
print("="*60)
correct_df = test_results_df[test_results_df['correct'] == True]
incorrect_df = test_results_df[test_results_df['correct'] == False]
print(f"Average confidence (correct predictions):   {correct_df['confidence'].mean():.4f}")
print(f"Average confidence (incorrect predictions): {incorrect_df['confidence'].mean():.4f}")

# Save predictions
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_path = f"/content/drive/MyDrive/test_predictions_{timestamp}.csv"
test_results_df.to_csv(save_path, index=False)
print(f"\nPredictions saved to: {save_path}")

# Show misclassifications with probabilities
misclassified = test_results_df[test_results_df['label'] != test_results_df['prediction']]
if len(misclassified) > 0:
    print("\n" + "="*60)
    print(f"Misclassifications: {len(misclassified)}/{len(test_df)}")
    print("="*60)
    print("\nFirst 10 examples:")
    for idx, row in misclassified.head(10).iterrows():
        print(f"\nStatement: {row['text']}")
        print(f"True: {row['label']}, Predicted: {row['prediction']}")
        print(f"P(0)={row['prob_0']:.4f}, P(1)={row['prob_1']:.4f}, Confidence={row['confidence']:.4f}")

In [ ]:
import json

# -----------------------------
# Save metrics in structured format for later use
# -----------------------------

# Create metrics dictionary
metrics_dict = {
    'model_name': 'llama-3b-instruct-arabic',
    'timestamp': timestamp,
    'dataset_info': {
        'total_samples': len(test_df),
        'class_0_samples': int((test_results_df['label'] == 0).sum()),
        'class_1_samples': int((test_results_df['label'] == 1).sum()),
    },
    'metrics': {
        'accuracy': float(accuracy),
        'precision': float(precision),
        'recall': float(recall),
        'f1_score': float(f1),
    },
    'confusion_matrix': {
        'true_negative': int(cm[0][0]),   # Actual 0, Predicted 0
        'false_positive': int(cm[0][1]),  # Actual 0, Predicted 1
        'false_negative': int(cm[1][0]),  # Actual 1, Predicted 0
        'true_positive': int(cm[1][1]),   # Actual 1, Predicted 1
    },
    'per_class_metrics': {
        'class_0': {
            'precision': float(precision_score(test_results_df['label'], predictions, pos_label=0)),
            'recall': float(recall_score(test_results_df['label'], predictions, pos_label=0)),
            'f1_score': float(f1_score(test_results_df['label'], predictions, pos_label=0)),
        },
        'class_1': {
            'precision': float(precision),  # Already calculated for class 1
            'recall': float(recall),
            'f1_score': float(f1),
        }
    },
    'confidence_analysis': {
        'avg_confidence_correct': float(correct_df['confidence'].mean()),
        'avg_confidence_incorrect': float(incorrect_df['confidence'].mean()),
    }
}

# Save as JSON (complete detail)
json_path = f"/content/drive/MyDrive/metrics_llama3b_{timestamp}.json"
with open(json_path, 'w') as f:
    json.dump(metrics_dict, f, indent=2)
print(f"Metrics JSON saved to: {json_path}")

# Save as single-row CSV (easy to combine with other models)
metrics_csv = pd.DataFrame([{
    'model': 'llama-3b-instruct-arabic',
    'timestamp': timestamp,
    'total_samples': len(test_df),
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1,
    'TN': cm[0][0],
    'FP': cm[0][1],
    'FN': cm[1][0],
    'TP': cm[1][1],
    'class_0_precision': precision_score(test_results_df['label'], predictions, pos_label=0),
    'class_0_recall': recall_score(test_results_df['label'], predictions, pos_label=0),
    'class_0_f1': f1_score(test_results_df['label'], predictions, pos_label=0),
    'class_1_precision': precision,
    'class_1_recall': recall,
    'class_1_f1': f1,
    'avg_conf_correct': correct_df['confidence'].mean(),
    'avg_conf_incorrect': incorrect_df['confidence'].mean(),
}])

csv_path = f"/content/drive/MyDrive/metrics_llama3b_{timestamp}.csv"
metrics_csv.to_csv(csv_path, index=False)
print(f"Metrics CSV saved to: {csv_path}")

# -----------------------------
# Helper function to generate LaTeX table later
# -----------------------------
def create_latex_table(csv_files_list):
    """
    Combine multiple model metrics CSVs into a LaTeX table.

    Args:
        csv_files_list: List of paths to metrics CSV files

    Returns:
        String containing LaTeX table code
    """
    # Combine all CSVs
    dfs = [pd.read_csv(f) for f in csv_files_list]
    combined_df = pd.concat(dfs, ignore_index=True)

    # Select key columns for the table
    table_df = combined_df[['model', 'accuracy', 'precision', 'recall', 'f1_score']]

    # Round to 3 decimal places
    for col in ['accuracy', 'precision', 'recall', 'f1_score']:
        table_df[col] = table_df[col].round(3)

    # Generate LaTeX
    latex_str = table_df.to_latex(index=False, escape=False,
                                   column_format='l|cccc',
                                   caption='Model Performance Comparison',
                                   label='tab:model_comparison')

    return latex_str

# Example usage (you'll do this later when you have multiple models):
# latex_table = create_latex_table([
#     '/path/to/metrics_llama3b_20241125_120000.csv',
#     '/path/to/metrics_xlmr_20241125_130000.csv',
#     '/path/to/metrics_other_model.csv'
# ])
# print(latex_table)